In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

import time
import numpy as np
from dask import delayed
from dask.distributed import Client
client = Client(processes=True, threads_per_worker=1,n_workers=2)

/tmp/ipykernel_506036/63027641.py:20: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
distributed.diskutils - INFO - Found stale lock file and directory '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/preproc/Maria/dask-worker-space/worker-p40s7e11', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/preproc/Maria/dask-worker-space/worker-26l4vcnu', purging


In [2]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'

haiyan_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_v = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['v'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_w = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['w'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_qv = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['qv'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_theta = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['theta'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_heatsum = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['heatsum'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_hdia = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['hdia'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_rad = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['rad'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:41<00:00,  8.28s/it]


In [3]:
haiyan_lw = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix+'_radcomp')['LW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_sw = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix+'_radcomp')['SW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.97s/it]


In [4]:
def find_start(haiyan_data=None,ref1=0,testexp=1):
    for i in range(len(haiyan_data[ref1])):
        if str((haiyan_data[ref1][i]==haiyan_data[testexp][i]).all())=='True':
            continue
        else:
            break
    return i

find_start(haiyan_lw,0,1)

25

In [5]:
def long_MariaExps(array=None,start=None):
    haiyan_temparray = [array[0][12:,:],array[1][int(start[0]):,:],array[2][int(start[1]):,:],array[3][int(start[2]):,:],array[4][int(start[3]):,:]]
    del array
    gc.collect()
    return np.concatenate(haiyan_temparray,axis=0)

start = [find_start(haiyan_lw,0,1),find_start(haiyan_lw,0,2),find_start(haiyan_lw,0,3),find_start(haiyan_lw,0,4)]
haiyan_ua,haiyan_va,haiyan_wa = long_MariaExps(haiyan_u,start),long_MariaExps(haiyan_v,start),long_MariaExps(haiyan_w,start)
haiyan_qva,haiyan_thetaa,haiyan_heatsuma = long_MariaExps(haiyan_qv,start),long_MariaExps(haiyan_theta,start),long_MariaExps(haiyan_heatsum,start)
haiyan_hdiaa,haiyan_swa,haiyan_lwa = long_MariaExps(haiyan_hdia,start),long_MariaExps(haiyan_sw,start),long_MariaExps(haiyan_lw,start)

In [34]:
def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

dims = ['sample','flatarray']
coords = dict(sample=np.linspace(0,haiyan_ua.shape[0]-1,haiyan_ua.shape[0]),flatarray=np.linspace(0,haiyan_ua.shape[1]-1,haiyan_ua.shape[1]))

ds = xr.Dataset(coords=coords)
maria_data=build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','lw','qv','sw'],\
                                   varfile=[haiyan_ua,haiyan_va,haiyan_wa,haiyan_thetaa,haiyan_heatsuma,haiyan_hdiaa,haiyan_lwa,haiyan_qva,haiyan_swa],dims=dims,coords=coords)
del haiyan_ua,haiyan_va,haiyan_wa,haiyan_thetaa,haiyan_heatsuma,haiyan_hdiaa,haiyan_lwa,haiyan_swa
gc.collect()

0

In [35]:
from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [36]:
pca_store = {}
for varnamex in (['u','v','w','theta','heatsum','hdia','lw','qv','sw']):#['u','v','w','theta','heatsum','hdia','rad','qv']):
    pca_store[varnamex] = fit_cheap_pca(n_batches=3,n_comps=150,var=maria_data[varnamex])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [07:19<00:00, 146.52s/it]


In [37]:
! pwd

/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/preproc/Maria


In [38]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/'
read_and_proc.save_to_pickle(folderpath+'PCA/PCAdict3D',pca_store,'PICKLE')

#### Intermediate

In [40]:
def long_MariaExps(array=None,start=None):
    haiyan_temparray = [array[0][12:,:],array[1][int(start[0]):,:],array[2][int(start[1]):,:],array[3][int(start[2]):,:],array[4][int(start[3]):,:]]
    del array
    gc.collect()
    return np.concatenate(haiyan_temparray,axis=0)

def to_azim(array=None,shape=[39,360,200]):
    def _to_azim(array=None):
        arrayn = array.reshape(array.shape[0],shape[0],shape[1],shape[2])
        return np.nanmean(arrayn,axis=2).reshape(array.shape[0],shape[0]*shape[2])
    return [_to_azim(obj) for obj in array]

def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [42]:
haiyan_uazim,haiyan_vazim,haiyan_wazim = long_MariaExps(to_azim(haiyan_u),start),long_MariaExps(to_azim(haiyan_v),start),long_MariaExps(to_azim(haiyan_w),start)
haiyan_qvazim,haiyan_thetaazim,haiyan_heatsumazim = long_MariaExps(to_azim(haiyan_qv),start),long_MariaExps(to_azim(haiyan_theta),start),long_MariaExps(to_azim(haiyan_heatsum),start)
haiyan_hdiaazim,haiyan_lwazim,haiyan_swazim = long_MariaExps(to_azim(haiyan_hdia),start),long_MariaExps(to_azim(haiyan_lw),start),long_MariaExps(to_azim(haiyan_sw),start)

In [43]:
dims = ['sample','flatarray']
coords = dict(sample=np.linspace(0,haiyan_uazim.shape[0]-1,haiyan_uazim.shape[0]),flatarray=np.linspace(0,haiyan_uazim.shape[1]-1,haiyan_uazim.shape[1]))

ds = xr.Dataset(coords=coords)
maria_data=build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','lw','qv','sw'],\
                                   varfile=[haiyan_uazim,haiyan_vazim,haiyan_wazim,haiyan_thetaazim,haiyan_heatsumazim,haiyan_hdiaazim,haiyan_lwazim,haiyan_qvazim,haiyan_swazim],dims=dims,coords=coords)
del haiyan_uazim,haiyan_vazim,haiyan_wazim,haiyan_thetaazim,haiyan_heatsumazim,haiyan_hdiaazim,haiyan_lwazim,haiyan_swazim
gc.collect()

0

In [44]:
from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [45]:
pca_store = {}
for varnamex in (['u','v','w','theta','heatsum','hdia','lw','qv','sw']):#['u','v','w','theta','heatsum','hdia','rad','qv']):
    pca_store[varnamex] = fit_cheap_pca(n_batches=3,n_comps=150,var=maria_data[varnamex])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.47it/s]


In [46]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
read_and_proc.save_to_pickle(folderpath+'PCA/PCAdict',pca_store,'PICKLE')

In [47]:
start

[25, 49, 85, 25]